Concept: first two subfigs in row 1, second two subfigs in row 2
* subfig 1) ear model with reconstructed surface ref: http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Poisson-surface-reconstruction
* subfig 2) point cloud of the model's surface with oriented normals (normals are encoded to RGB color values) ref: https://www.mathworks.com/matlabcentral/fileexchange/71178-normal-vector-to-rgb
* subfig 3) point cloud of the model's surface with APD values encoded into RGB color values (TE)
* subfig 4) point cloud of the model's surface with APD values encoded into RGB color values (TM)

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import set_colorblind, save_fig, scatter_3d, fig_config, colormap_from_array
from helpers import (ABSTRACT_ID, clean_df, export_pcd, export_fields,
                     poynting_vector, estimate_normals, get_imcolors,
                     normals_to_rgb)

## Subfig a

In [ ]:
mode = 'tm'
frequency = 60
df = load_ear_data('tm', 60)
df = clean_df(df)
xyz = export_pcd(df)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
pcd.paint_uniform_color([0.5, 0.5, 0.5])
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
pcd.estimate_normals()
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

In [ ]:
view_config_a = {
    'zoom': 0.69999999999999996,
    'front': [0.92231085610160646, 0.17546582462905541, 0.34431733779228646],
    'lookat': [71.236805645281521, 22.531933429935712, -8.12589641784127],
    'up': [-0.16595758534247468, 0.98447554162242001, -0.057148821637356101],
}
color = get_imcolors([cframe, pcd, rec_mesh], view_config_a)
set_colorblind()
fig_config()
fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(color, origin='upper')

fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_1a')
save_fig(fig, fname=fname, formats=['png'])

## Subfig b

In [ ]:
mode = 'tm'
frequency = 60
df = load_ear_data('tm', 60)
df = clean_df(df)
xyz = export_pcd(df)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
skip = 30
pcd_dwnsmpd = pcd.uniform_down_sample(skip)

In [ ]:
try:
    normals = np.load(os.path.join('analysis', f'normals_{skip}.npy'))
    pcd_dwnsmpd.normals = o3d.utility.Vector3dVector(normals)
except:
    pcd_dwnsmpd.estimate_normals()
    pcd_dwnsmpd.orient_normals_consistent_tangent_plane(k=30)
    normals = np.asarray(pcd_dwnsmpd.normals)
    np.save(os.path.join('analysis', f'normals_{skip}'), normals)

In [ ]:
view_config_b = {
    'zoom': 0.69999999999999996,
    'front': [0.88361503789438389, 0.001013610506472548, -0.46821302566315681],
    'lookat': [67.638223693668408, 17.049614524433402, -12.088523009149476],
    'up': [-0.013882221702813466, 0.99961473638476039, -0.024034615100309641],
}
# quiver plot version
pcd_dwnsmpd.paint_uniform_color(np.array([0.6, 0.6, 0.6]))
color = get_imcolors([cframe, pcd_dwnsmpd], view_config_b, point_show_normal=True)
# normal vectors mapped to RGB cube
# pcd_dwnsmpd.colors = o3d.utility.Vector3dVector(normals_to_rgb(normals))
# color = get_imcolors([cframe, pcd_dwnsmpd], view_config_b, point_show_normal=False)
set_colorblind()
fig_config()
fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(color, origin='upper')

fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_1b')
save_fig(fig, fname=fname, formats=['png'])

## Subfig c and d

In [ ]:
mode = 'te'
frequency = 60
df = load_ear_data('te', 60)
df = clean_df(df)
xyz = export_pcd(df)

In [ ]:
E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

crop_idxs = np.where(xyz[:, 0] > 67.5)[0]
xyz_crop = xyz[crop_idxs]
n_crop = normals[crop_idxs]

pcd_crop = o3d.geometry.PointCloud()
pcd_crop.points = o3d.utility.Vector3dVector(xyz_crop)
center_crop = pcd_crop.get_center()

xyz_crop_t = np.c_[xyz_crop[:, 0] - center_crop[0],
                   xyz_crop[:, 1] - center_crop[1],
                   xyz_crop[:, 2] - center_crop[2]]

Sx_crop, Sy_crop, Sz_crop = Sx[crop_idxs], Sy[crop_idxs], Sz[crop_idxs]
Sr_crop = abs(Sx_crop.real * n_crop[:, 0]
              + Sy_crop.real * n_crop[:, 1]
              + Sz_crop.real * n_crop[:, 2])

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.75, text_size=20)
Sr_label = '$APD$'
fig, ax = scatter_3d({'$z$ [mm]': xyz_crop_t[:, 2],
                      '$x$ [mm]': xyz_crop_t[:, 0],
                      '$y$ [mm]': xyz_crop_t[:, 1],
                      Sr_label: Sr_crop},
                     elev=[15], azim=[150])
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis.labelpad = 12
ax.yaxis.labelpad = 12
ax.zaxis.labelpad = 5

if mode == 'te':
    subfig_id = 'c'
else:
    subfig_id = 'd'
fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_1{subfig_id}')
save_fig(fig, fname=fname, formats=['png'])